In [4]:
# Compressed Colab handwritten digit recognizer — keeps original logic.
# Put a GitHub raw .h5 URL into MODEL_URLS if you want Colab to fetch it.
import os, io, base64, json, requests
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from IPython.display import HTML, display

MODEL_PATH = "/content/digit_model.h5"
MODEL_URLS = [
    "https://github.com/praxavv/Neural-Network/raw/refs/heads/main/Digit-Recognition/digit_model.h5"
]

def _download_model(urls, target=MODEL_PATH):
    for u in urls:
        try:
            print(f"downloading model from {u} ...")
            r = requests.get(u, stream=True, timeout=30)
            r.raise_for_status()
            with open(target, "wb") as f:
                for chunk in r.iter_content(1 << 20):
                    f.write(chunk)
            print("download complete.")
            return target
        except Exception as e:
            print("download failed:", e)
    return None

class Recognizer:
    def __init__(self, path=MODEL_PATH, urls=MODEL_URLS):
        self.path = path
        if not os.path.exists(self.path) and urls:
            _download_model(urls, self.path)
        if not os.path.exists(self.path):
            raise FileNotFoundError(f"No model at {self.path}. Set MODEL_URLS or upload file.")
        print("loading model...", end=" ")
        self.model = keras.models.load_model(self.path)
        print("done.")

    def predict_b64(self, data_uri):
        try:
            b = base64.b64decode(data_uri.split(",",1)[1])
            img = Image.open(io.BytesIO(b)).convert("L")
            resample = getattr(Image, "Resampling", Image).LANCZOS if hasattr(Image, "Resampling") or hasattr(Image, "LANCZOS") else Image.ANTIALIAS
            img = img.resize((28,28), resample)
            arr = 255 - np.array(img).astype("float32")  # invert (canvas vs MNIST)
            arr = arr / 255.0
            arr = arr.reshape(1,28,28,1)
            pred = self.model.predict(arr, verbose=0)[0]
            digit = int(np.argmax(pred))
            conf = float(np.max(pred))
            return {"digit": digit, "confidence": conf, "all_predictions": pred.tolist()}
        except Exception as e:
            return {"error": str(e)}

# instantiate (will try to download if MODEL_URLS provided)
recognizer = Recognizer()

# Minimal JS + HTML interface (keeps same behavior)
js = r"""
<script>
function isCanvasEmpty(c){
  const d=c.getContext('2d').getImageData(0,0,c.width,c.height).data;
  for(let i=0;i<d.length;i+=4) if(d[i]<255||d[i+1]<255||d[i+2]<255) return false;
  return true;
}
function makePrediction(){
  const c=document.getElementById('drawingCanvas');
  if(isCanvasEmpty(c)){document.getElementById('result').innerHTML='✏️ Please draw a digit first!';return;}
  document.getElementById('result').innerHTML='🔄 Predicting...';
  const img=c.toDataURL('image/png');
  google.colab.kernel.invokeFunction('predict_digit',[img], {});
}
function clearCanvas(){
  const c=document.getElementById('drawingCanvas'), ctx=c.getContext('2d');
  ctx.fillStyle='white'; ctx.fillRect(0,0,c.width,c.height);
  document.getElementById('result').innerHTML='✏️ Canvas cleared! Draw a digit (0-9)';
  document.getElementById('confidence').innerHTML=''; document.getElementById('allPredictions').innerHTML='';
}
function updateResult(d,conf,all){
  document.getElementById('result').innerHTML=`🎯 Predicted Digit: <strong>${d}</strong>`;
  document.getElementById('confidence').innerHTML=`Confidence: ${(conf*100).toFixed(1)}%`;
  let html='<div style="margin-top:10px;"><strong>All Predictions:</strong>';
  for(let i=0;i<all.length;i++){let p=(all[i]*100).toFixed(1); html+=`<div style="display:flex;align-items:center"><span style="width:24px">${i}:</span><div style="background:#3498db;height:14px;width:${Math.max(p*1.5,2)}px;margin:0 8px"></div><small>${p}%</small></div>`}
  html+='</div>'; document.getElementById('allPredictions').innerHTML=html;
}
/* simple drawing */
let drawing=false,last=[0,0];
function setupCanvas(){
  const c=document.getElementById('drawingCanvas'), ctx=c.getContext('2d');
  ctx.lineCap='round'; ctx.lineJoin='round'; ctx.lineWidth=15; ctx.strokeStyle='black';
  ctx.fillStyle='white'; ctx.fillRect(0,0,c.width,c.height);
  function pos(e){const r=c.getBoundingClientRect(), t=(e.touches?e.touches[0]:e); return [t.clientX-r.left,t.clientY-r.top];}
  c.addEventListener('pointerdown', e=>{drawing=true; last=pos(e)});
  c.addEventListener('pointermove', e=>{ if(!drawing) return; const p=pos(e); ctx.beginPath(); ctx.moveTo(...last); ctx.lineTo(...p); ctx.stroke(); last=p;});
  c.addEventListener('pointerup', ()=>drawing=false);
}
setTimeout(setupCanvas,100);
</script>
"""

html = f"""
<style>
body{{font-family:Segoe UI,system-ui;max-width:900px;margin:0 auto;padding:18px;background:linear-gradient(135deg,#667eea,#764ba2);color:#222}}
.container{{background:#fff;border-radius:12px;padding:20px;box-shadow:0 8px 24px rgba(0,0,0,.2)}}
#drawingCanvas{{border:3px solid #3498db;border-radius:10px;background:white;touch-action:none}}
.buttons{{text-align:center;margin-top:12px}}
button{{padding:10px 18px;border-radius:20px;border:none;cursor:pointer}}
.predict{{
  background:linear-gradient(45deg,#3498db,#2980b9);color:#fff;margin-right:8px
}}
.clear{{
  background:linear-gradient(45deg,#e74c3c,#c0392b);color:#fff
}}
.results{{margin-top:12px;padding:12px;border-radius:10px;background:linear-gradient(135deg,#f8f9fa,#e9ecef)}}
</style>
<div class="container">
  <h1 style="color:black;">✍️ Handwritten Digit Recognition</h1>
  <div>Draw a digit (0-9) and click Predict.</div>
  <div style="text-align:center;margin:18px 0">
    <canvas id="drawingCanvas" width="400" height="400"></canvas>
  </div>
  <div class="buttons">
    <button class="predict" onclick="makePrediction()">🔮 Predict Digit</button>
    <button class="clear" onclick="clearCanvas()">🗑️ Clear Canvas</button>
  </div>
  <div class="results">
    <div id="result">✏️ Draw a digit to get started!</div>
    <div id="confidence"></div>
    <div id="allPredictions"></div>
  </div>
</div>
{js}
"""

# Python callback used by the JS
def predict_digit(data_uri):
    r = recognizer.predict_b64(data_uri)
    if "error" in r:
        code = f"document.getElementById('result').innerHTML='❌ Error: {r['error']}';document.getElementById('confidence').innerHTML='';document.getElementById('allPredictions').innerHTML='';"
    else:
        code = f"updateResult({r['digit']},{r['confidence']},{json.dumps(r['all_predictions'])});"
    display(HTML(f"<script>{code}</script>"))

# register callback (if in Colab)
try:
    from google.colab import output
    output.register_callback('predict_digit', predict_digit)
except Exception:
    print("⚠️ Not in Colab — callback registration skipped.")

display(HTML(html))


loading model... done.
